# Sistema RAG Completo - Llama 3.1 8B

Este notebook combina a construção do índice e a execução do sistema RAG com avaliação de métricas.

## Estrutura:
1. **Configuração e Imports**
2. **Construção do Índice** (executar 1x e salvar)
3. **Carregamento do Sistema RAG** (carregar modelos salvos)
4. **Execução de Perguntas** (rodar N vezes)
5. **Avaliação de Métricas** (análise separada dos resultados)

---
## 1. Configuração e Imports

In [1]:
%%writefile requirements.txt

# --- Índices para pacotes pré-compilados com CUDA 12.4 ---
--extra-index-url https://download.pytorch.org/whl/cu126
--extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124

# --- Versões Fixas para Estabilidade ---

# Core de IA e Machine Learning
torch
torchvision
torchaudio
#numpy
#pandas

# Bibliotecas do RAG
langchain
langchain-community
langchain-core
sentence-transformers
faiss-gpu-cu12
llama-cpp-python

# Ferramentas e Utilitários
PyMuPDF
unstructured[pdf]==0.18.15
huggingface_hub
evaluate
rouge_score
tqdm
poppler-utils
bert_score



Overwriting requirements.txt


In [ ]:
!pip install -r requirements.txt

In [ ]:
import os
import shutil
import torch
import pickle
import re
import gc
import pandas as pd
import pymupdf
from tqdm import tqdm
from collections import OrderedDict
from langchain_core.documents import Document
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever
from langchain.prompts import PromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp
from langchain.chains import LLMChain
from huggingface_hub import hf_hub_download
from sentence_transformers import CrossEncoder
import evaluate

print("✓ Imports realizados com sucesso!")
print(f"Device disponível: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

✓ Imports realizados com sucesso!
Device disponível: CUDA


In [ ]:
# PARÂMETROS GLOBAIS
DOCS_PATH = "documentos/"
VECTORSTORE_PATH = "faiss_final_index"
DOCSTORE_PATH = "final_docstore"
MODEL_EMBEDDING_NAME = "BAAI/bge-m3"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Configuração do Modelo Llama 3.1
MODEL_REPO_ID = "lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF"
MODEL_BASENAME = "Meta-Llama-3.1-8B-Instruct-Q5_K_M.gguf"
LLM_N_GPU_LAYERS = -1
LLM_N_CTX = 8192
RETRIEVER_K = 10
TOP_K_AFTER_RERANK = 4
CROSS_ENCODER_MODEL = 'cross-encoder/ms-marco-MiniLM-L-6-v2'

# verificar e criar pasta DOCS_PATH
if not os.path.exists(DOCS_PATH):
    os.makedirs(DOCS_PATH)
    print(f"Pasta '{DOCS_PATH}' criada com sucesso.")
else:
    print(f"A pasta '{DOCS_PATH}' já existe.")

print("✓ Parâmetros configurados!")


A pasta 'documentos/' já existe.
✓ Parâmetros configurados!


---
## 2. Construção do Índice FAISS
**⚠️ Execute esta seção apenas UMA VEZ ou quando atualizar os documentos**

In [ ]:
def cleanup_old_index(vectorstore_path, docstore_path):
    """Limpa os diretórios antigos do índice e do docstore."""
    for path in [vectorstore_path, docstore_path]:
        if os.path.exists(path):
            shutil.rmtree(path)
    os.makedirs(vectorstore_path, exist_ok=True)
    os.makedirs(docstore_path, exist_ok=True)
    print("Limpeza e recriação das pastas concluída.")

def load_and_clean_definitively(folder_path: str) -> list:
    """
    Solução v3: Processa cada PDF, une todas as suas páginas em um
    único 'page_content' para permitir o chunking através das fronteiras
    das páginas, e armazena um 'page_map' nos metadados para
    permitir a localização da página original.
    """
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith(".pdf")]
    all_documents = [] # Lista de Documentos (um por PDF)

    for filename in tqdm(pdf_files, desc="Processando PDFs (v3)"):
        file_path = os.path.join(folder_path, filename)

        full_text_content = ""
        # page_map armazena tuplas de: (indice_final_char, numero_pagina)
        page_map = []
        current_char_index = 0

        with pymupdf.open(file_path) as doc:
            for page_num, page in enumerate(doc):
                page_height = page.rect.height
                margin_top = page_height * 0.08
                margin_bottom = page_height * 0.92

                blocks = page.get_text("blocks")
                valid_blocks = [b for b in blocks if margin_top < b[1] < margin_bottom]
                valid_blocks.sort(key=lambda b: b[1])

                page_text = " ".join([b[4].replace('\n', ' ') for b in valid_blocks])
                page_text = re.sub(r'\s+', ' ', page_text).strip()

                if page_text:
                    # Adiciona um espaço entre as páginas para garantir a separação
                    if current_char_index > 0:
                        page_text = " " + page_text

                    full_text_content += page_text
                    current_char_index = len(full_text_content)

                    # O 'page_map' armazena o índice do *último* caractere desta página
                    page_map.append((current_char_index, page_num + 1))

        if full_text_content:
            all_documents.append(Document(
                page_content=full_text_content,
                metadata={
                    "source": os.path.splitext(filename)[0],
                    "page_map": page_map # Armazena o mapa de páginas
                }
            ))

    print(f"Limpeza v3 concluída. {len(all_documents)} documentos (PDFs) processados.")
    return all_documents

print("✓ Funções de processamento definidas!")

In [ ]:
# EXECUTAR APENAS QUANDO NECESSÁRIO RECONSTRUIR O ÍNDICE
print("Iniciando construção do índice...\n")

cleanup_old_index(VECTORSTORE_PATH, DOCSTORE_PATH)
docs = load_and_clean_definitively(DOCS_PATH)

# Splitters para a estratégia Parent/Child
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200, add_start_index=True)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

embeddings = HuggingFaceEmbeddings(model_name=MODEL_EMBEDDING_NAME, model_kwargs={'device': DEVICE})

vectorstore = FAISS.from_texts(texts=["_INITIALIZING_"], embedding=embeddings)
vectorstore.delete(list(vectorstore.index_to_docstore_id.values()))

store = InMemoryStore()

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore, docstore=store,
    child_splitter=child_splitter, parent_splitter=parent_splitter,
)

print("Adicionando documento limpo ao índice...")
retriever.add_documents(docs, ids=None)

vectorstore.save_local(VECTORSTORE_PATH)
with open(os.path.join(DOCSTORE_PATH, "store.pkl"), "wb") as f:
    pickle.dump(store, f)

print(f"\n✓ Índice definitivo salvo em '{VECTORSTORE_PATH}' e '{DOCSTORE_PATH}'")

In [ ]:
def save_faiss_index_content(retriever, output_file="faiss_index_content.txt"):
    """
    Recupera todos os documentos do índice FAISS (através do retriever)
    e salva o conteúdo em um arquivo de texto estruturado.
    """
    print(f"Salvando conteúdo do índice FAISS em '{output_file}'...")
    try:
        # Como o retriever usa o docstore, podemos iterar sobre ele.
        # No entanto, não há um método direto para obter TODOS os documentos.
        # Uma abordagem é recuperar documentos para uma query genérica e depois
        # tentar recuperar todos os documentos únicos do docstore.
        # Uma forma mais robusta é acessar diretamente o docstore se ele for InMemoryStore

        if isinstance(retriever.docstore, InMemoryStore):
            all_docs = list(retriever.docstore.yield_keys())
            # Para cada key (ID do documento pai), recuperar o documento pai
            # e seus filhos, se houver.
            with open(output_file, "w", encoding="utf-8") as f:
                for doc_id in tqdm(all_docs, desc="Escrevendo documentos no arquivo"):
                    doc = retriever.docstore.mget([doc_id])[0] # Recupera o documento pai
                    f.write(f"--- Document ID: {doc_id} ---\n")
                    f.write(f"Content:\n{doc.page_content}\n")
                    # Adicionar metadados se existirem
                    if doc.metadata:
                        f.write("Metadata:\n")
                        for key, value in doc.metadata.items():
                            f.write(f"  {key}: {value}\n")
                    f.write("-" * 20 + "\n\n")
            print("\n✓ Conteúdo do índice FAISS salvo com sucesso!")
        else:
            print("Erro: O docstore não é InMemoryStore. Não é possível salvar o conteúdo diretamente.")

    except Exception as e:
        print(f"Erro ao salvar o conteúdo do índice FAISS: {e}")

# Exemplo de uso (descomente para rodar):
save_faiss_index_content(retriever)

---
## 3. Carregamento do Sistema RAG
**Execute esta seção para carregar os índices e modelos já salvos**

In [ ]:
# Carregar embeddings
print("Carregando modelo de embeddings...")
embeddings = HuggingFaceEmbeddings(
    model_name=MODEL_EMBEDDING_NAME,
    model_kwargs={'device': DEVICE}
)
print("✓ Embeddings carregados!")

Carregando modelo de embeddings...


/tmp/ipython-input-3050706894.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✓ Embeddings carregados!


In [ ]:
# Carregar vectorstore FAISS
print("Carregando índice FAISS...")
vectorstore = FAISS.load_local(
    VECTORSTORE_PATH,
    embeddings,
    allow_dangerous_deserialization=True
)
print("✓ Vectorstore carregado!")

Carregando índice FAISS...
✓ Vectorstore carregado!


In [ ]:
# Carregar docstore
print("Carregando docstore...")
with open(os.path.join(DOCSTORE_PATH, "store.pkl"), "rb") as f:
    store = pickle.load(f)

child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter
)
retriever.search_kwargs['k'] = RETRIEVER_K
print("✓ Retriever configurado!")

Carregando docstore...
✓ Retriever configurado!


In [ ]:
# Carregar cross-encoder
print("Carregando cross-encoder...")
cross_encoder = CrossEncoder(CROSS_ENCODER_MODEL)
print("✓ Cross-encoder carregado!")

Carregando cross-encoder...
✓ Cross-encoder carregado!


In [ ]:
# Baixar e carregar LLM
print(f"Baixando modelo: {MODEL_BASENAME}...")
model_path = hf_hub_download(repo_id=MODEL_REPO_ID, filename=MODEL_BASENAME)
print(f"✓ Modelo baixado: {model_path}")

print("\nInicializando LLM...")
llm = LlamaCpp(
    model_path=model_path,
    n_gpu_layers=LLM_N_GPU_LAYERS,
    n_ctx=LLM_N_CTX,
    max_tokens=512,
    repeat_penalty=1.15,
    temperature=0.1,
    top_p=0.9,
    top_k=40,
    n_batch=512,
    f16_kv=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    verbose=False
)
print("✓ LLM carregado!")

Baixando modelo: Meta-Llama-3.1-8B-Instruct-Q5_K_M.gguf...
✓ Modelo baixado: /root/.cache/huggingface/hub/models--lmstudio-community--Meta-Llama-3.1-8B-Instruct-GGUF/snapshots/8601e6db71269a2b12255ebdf09ab75becf22cc8/Meta-Llama-3.1-8B-Instruct-Q5_K_M.gguf

Inicializando LLM...


llama_context: n_ctx_per_seq (8192) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


✓ LLM carregado!


In [ ]:
# Criar chain RAG
template = (
    "<|start_header_id|>system<|end_header_id|>\n\n"
    "Você é um assistente especializado em responder perguntas com base em documentos acadêmicos.\n\n"
    "INSTRUÇÕES:\n"
    "- Leia atentamente o contexto fornecido\n"
    "- Responda de forma completa e natural, como em uma conversa\n"
    "- Use APENAS informações presentes no contexto\n"
    "- Para perguntas sobre 'quando', forneça a resposta em uma frase completa mencionando o ano e o local\n"
    "- Seja preciso mas não robotizado\n"
    "- Evite falar 'De acordo com o contexto fornecido'\n"
    "- Evite respostas de uma única palavra ou número<|eot_id|>"
    "<|start_header_id|>user<|end_header_id|>\n\n"
    "Contexto:\n{context}\n\n"
    "Pergunta: {question}<|eot_id|>"
    "<|start_header_id|>assistant<|end_header_id|>\n\n"
)

prompt = PromptTemplate(template=template, input_variables=["context", "question"])
rag_chain = LLMChain(prompt=prompt, llm=llm)

print("✓ RAG Chain criada!")
print("\n" + "="*80)
print("SISTEMA RAG PRONTO PARA USO!")
print("="*80)

✓ RAG Chain criada!

SISTEMA RAG PRONTO PARA USO!


/tmp/ipython-input-3309389960.py:20: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  rag_chain = LLMChain(prompt=prompt, llm=llm)


---
## 4. Funções de Execução de Perguntas

In [ ]:
def rerank_documents(question, retrieved_docs, cross_encoder, top_k):
    """Reordena os documentos usando um CrossEncoder."""
    pairs = [[question, doc.page_content] for doc in retrieved_docs]
    scores = cross_encoder.predict(pairs)
    doc_scores = list(zip(retrieved_docs, scores))
    doc_scores_sorted = sorted(doc_scores, key=lambda x: x[1], reverse=True)

    print(f"\n[DEBUG] Top {top_k} documentos após re-ranking:")
    for i, (doc, score) in enumerate(doc_scores_sorted[:top_k], 1):
        preview = doc.page_content[:80].replace('\n', ' ')
        print(f"  #{i} (score: {score:.4f}): {preview}...")

    return [doc for doc, score in doc_scores_sorted[:top_k]]

def find_page_from_metadata(doc):
    """
    Função auxiliar para encontrar o número da página usando o
    'page_map' e 'start_index' do metadata do chunk.
    """
    if not doc.metadata:
        return "N/A", "N/A"

    source_file = doc.metadata.get("source", "N/A")
    page_map = doc.metadata.get("page_map")
    # 'start_index' é adicionado automaticamente pelo RecursiveCharacterTextSplitter
    start_index = doc.metadata.get("start_index")

    # Se 'page_map' ou 'start_index' não existirem, tenta o fallback para o método v2
    if page_map is None or start_index is None:
        page_num = doc.metadata.get("page", "N/A")
        return source_file, page_num

    # --- Lógica v3 ---
    # Encontra a primeira página cujo 'end_char_index' (índice final)
    # é MAIOR que o 'start_index' (índice inicial) do chunk.
    for end_char, page_num in page_map:
        if start_index < end_char:
            return source_file, page_num

    # Se não encontrar (ex: chunk começa após o fim do último mapa),
    # usa a última página do mapa como fallback.
    if page_map:
        return source_file, page_map[-1][1]

    return source_file, "N/A" # Fallback final

# MANTENHA a função find_page_from_metadata como está.
# SUBSTITUA APENAS a função ask por esta:

def ask(question, rag_chain, retriever, cross_encoder, verbose=True):
    """
    Pipeline v8: Retrieve -> Re-rank -> Generate (LLM) -> Group Sources -> Post-process.
    """
    if verbose:
        print(f"\n{'='*80}")
        print(f"PERGUNTA: {question}")
        print('='*80)

    retrieved_docs = retriever.invoke(question)
    if verbose:
        print(f"✓ Recuperados {len(retrieved_docs)} documentos")

    reranked_docs = rerank_documents(
        question, retrieved_docs, cross_encoder, TOP_K_AFTER_RERANK)

    # --- ETAPA 1: COLETAR FONTES (SEM INJETAR NO CONTEXTO) ---
    sources = []
    source_set = set()
    for doc in reranked_docs:
        source_file, page_num = find_page_from_metadata(doc)
        # Usamos um formato simples para facilitar o parsing
        citation_tuple = (source_file, page_num)
        if citation_tuple not in source_set:
            sources.append(citation_tuple)
            source_set.add(citation_tuple)

    # --- ETAPA 2: CONSTRUIR CONTEXTO LIMPO PARA O LLM ---
    context_parts = []
    for i, doc in enumerate(reranked_docs, 1):
        context_parts.append(f"[Trecho {i}]: {doc.page_content}")
    context = "\n\n".join(context_parts)

    if verbose:
        print(f"\n[DEBUG] Contexto LIMPO enviado ao LLM (primeiros 400 caracteres):")
        print("-" * 80)
        print(context[:400])
        print("-" * 80)

    # --- ETAPA 3: LLM GERA A RESPOSTA (FOCO TOTAL) ---
    result = rag_chain.invoke({"context": context, "question": question})
    answer = result['text'].strip()

    # Limpeza de tokens especiais
    tokens_to_remove = ["</s>", "<|eot_id|>", "<|end_header_id|>", "<|start_header_id|>"]
    for token in tokens_to_remove:
        answer = answer.replace(token, "")
    answer = answer.strip()

    # --- ETAPA 4: PÓS-PROCESSAMENTO (AGRUPAR E FORMATAR FONTES) ---
    clean_answer = answer.rstrip('.').strip()

    final_answer = f"{clean_answer}." # Default se não houver fontes

    if sources:
        # 1. Agrupar páginas por arquivo, mantendo a ordem de relevância
        grouped_sources = OrderedDict()
        for file_name, page_num in sources:
            if file_name not in grouped_sources:
                grouped_sources[file_name] = []
            # Adiciona a página se ainda não estiver na lista
            if page_num not in grouped_sources[file_name]:
                 grouped_sources[file_name].append(str(page_num)) # Converte p/ string

        # 2. Formatar a string de citação
        citation_parts = []
        for file_name, page_list in grouped_sources.items():
            # Une as páginas: ex: "17, 75, 76"
            page_str = ", ".join(page_list)
            # Formata: "PPC de ADS - Picos, p. 17, 75, 76"
            citation_parts.append(f"{file_name}, p. {page_str}")

        # Une os diferentes arquivos: "File1, p. 1; File2, p. 5"
        final_citation_str = "; ".join(citation_parts)

        # 3. Adiciona à resposta
        final_answer = f"{clean_answer} ({final_citation_str})."

    final_answer = final_answer.strip()

    if verbose:
        print(f"\n{'='*80}")
        print(f"RESPOSTA (com citação pós-processada e agrupada):")
        print(final_answer)
        print('='*80)

    # A função continua retornando apenas a resposta final e o contexto.
    return final_answer, context

print("✓ Funções de execução definidas!")

✓ Funções de execução definidas!


---
## 5. Dataset de Avaliação
**Adicione ou modifique as perguntas aqui**

In [ ]:
# Dataset de perguntas e respostas esperadas
evaluation_dataset = [
    {
        "pergunta": "Quando o curso de Tecnologia em Análise e Desenvolvimento de Sistemas foi implantado no Campus de Picos?",
        "resposta_esperada": "O curso foi implantado no Campus de Picos em 2013 (PPC de ADS - Picos, p. 17)."
    },
    {
        "pergunta": "Quais são as modalidades aceitas para a apresentação do TCC?",
        "resposta_esperada": "O TCC pode ser apresentado em formato de Monografia, Artigo Científico, Relatório Técnico de Software (RTS) ou Relatório Técnico de Trabalho/Estágio (RTT) (PPC de ADS - Picos, p. 75)."
    },
    {
        "pergunta": "Qual foi o montante total de investimento que o setor de TI no Brasil atingiu em 2021, englobando os mercados de software, serviços, hardware e exportações?",
        "resposta_esperada": "O investimento atingiu R$238,2 bilhões (US$ 46,2 bilhões) (PPC de ADS - Picos, p. 19)."
    },
    {
        "pergunta": "Qual o pré-requisito para cursar Programação para Dispositivos Móveis?",
        "resposta_esperada": "O pré-requisito para cursar Programação para Dispositivos Móveis é a disciplina de Programação Orientada a Objetos (PPC de ADS - Picos, p. 56)."
    },
    {
        "pergunta": "Quantas vagas anuais são ofertadas para o curso de ADS no campus Picos e qual o turno de funcionamento?",
        "resposta_esperada": "São ofertadas 40 vagas por ano para o turno vespertino (PPC de ADS - Picos, p. 23)."
    },
    {
        "pergunta": "Qual é a carga horária total do curso de ADS e como ela é dividida entre disciplinas obrigatórias e atividades complementares?",
        "resposta_esperada": "A carga horária total do curso é de 2100 horas, distribuídas em 2000 horas de disciplinas obrigatórias e 100 horas de atividades complementares (PPC de ADS - Picos, p. 38)."
    },
    {
        "pergunta": "Quais são os dois laboratórios de pesquisa e extensão disponíveis para os alunos do curso de ADS desenvolverem projetos?",
        "resposta_esperada": "Os alunos podem desenvolver projetos de pesquisa e extensão nos laboratórios Mambee (Fábrica Escola de Software) e LIARA (Laboratório de Inteligência Artificial, Robótica e Automação) (PPC de ADS - Picos, p. 100)."
    },
    {
        "pergunta": "Como é calculada a nota final do Trabalho de Conclusão de Curso (TCC)?",
        "resposta_esperada": "A nota do aluno orientando atenderá ao cálculo da média, conforme: Média = (50 x N1 + 25 x N2 + 25 x N3) / 100, Onde N1 é a nota do orientador (e/ou co-orientador) e N2 e N3 são as notas dos demais membros da banca. (PPC de ADS - Picos, p. 76)."
    },
    {
        "pergunta": "Qual disciplina é o pré-requisito necessário para cursar Redes de Computadores?",
        "resposta_esperada": "O pré-requisito para cursar a disciplina de Redes de Computadores é a disciplina de Introdução a Computação (PPC de ADS - Picos, p. 53)."
    },
    {
        "pergunta": "Em que ano a instituição, que hoje é o IFPI, foi efetivamente transformada em Centro Federal de Educação Tecnológica do Piauí (CEFET-PI)?",
        "resposta_esperada": "A transformação da instituição em Centro Federal de Educação Tecnológica do Piauí (CEFET-PI) foi efetivada em 1999 (PPC de ADS - Picos, p. 9)."
    },

    # # organização didática

    {
        "pergunta": "Qual é o número e a data da Resolução Normativa que atualiza e consolida a Organização Didática do IFPI?",
        "resposta_esperada": "A Resolução Normativa é a 111/2022-CONSUP/OSUPCOL/REI/IFPI, de 17 de março de 2022 (Organização Didática, p. 1)."
    },
    {
        "pergunta": "De acordo com o Art. 8º, qual é a definição de 'Dia letivo'?",
        "resposta_esperada": "Conforme o Art. 8º, § 4º, 'Dia letivo diz respeito ao dia de efetivo trabalho escolar com a participação discente e docente, constante no calendário escolar ou que a instituição readéque conforme necessidade', de acordo com a Lei nº 9.394/96 (LDB) e demais dispositivos legais (Organização Didática, p. 4)."
    },
    {
        "pergunta": "Qual o percentual mínimo de vagas que o IFPI deve garantir para os cursos de educação profissional técnica de nível médio, prioritariamente na forma de cursos integrados (inciso I do Art. 7º)?",
        "resposta_esperada": "Conforme o parágrafo único do Art. 7º, o IFPI deverá garantir o mínimo de 50% (cinquenta por cento) de suas vagas para atender aos objetivos definidos no inciso I deste artigo (Organização Didática, p. 3)."
    },
    {
        "pergunta": "Para quais tipos de curso o trancamento de matrícula é restrito, conforme o Art. 41?",
        "resposta_esperada": "De acordo com o parágrafo único do Art. 41, 'O trancamento de matrícula é restrito aos cursos superiores e técnicos concomitantes/subsequentes' (Organização Didática, p. 13)."
    },
    {
        "pergunta": "Quantas vezes um aluno pode ser reprovado consecutivamente em séries/módulos antes de ocorrer o cancelamento compulsório da matrícula, segundo o Art. 50?",
        "resposta_esperada": "O cancelamento compulsório ocorrerá por 'reprovação em séries/módulos por TRÊS (3) vezes CONSECUTIVAS', conforme o Art. 50, inciso V (Organização Didática, p. 15)."
    },
    {
        "pergunta": "Nos cursos técnicos integrados organizados em períodos semestrais, qual é a fórmula para calcular a Média Final Semestral (MFS) após o aluno realizar a Prova Final Semestral (PFS)?",
        "resposta_esperada": "A Média Final Semestral (MFS) é obtida pela média aritmética entre a Média Semestral (MS) e a Nota da Prova Final Semestral (PFS), dada pela fórmula: $MFS=\\frac{MS+PFS}{2}$ (Organização Didática, p. 18)."
    },
    {
        "pergunta": "Qual é a Média Anual (MA) mínima necessária para que um aluno de curso técnico integrado anual (forma seriada anual) seja submetido à Prova Final (PF)?",
        "resposta_esperada": "Será submetido a uma Prova Final (PF) o aluno que 'obtiver média anual igual ou superior a 4,0 (quatro) e inferior a 7,0 (sete)', em até 08 disciplinas (Organização Didática, p. 19)."
    },
    {
        "pergunta": "Qual o prazo, em dias úteis, para um aluno solicitar a verificação de aprendizagem em segunda chamada, conforme o Art. 107?",
        "resposta_esperada": "O aluno deve solicitar a segunda chamada 'no prazo de até 72 (setenta e duas) horas, considerando os dias úteis, após a realização da avaliação à qual não se fez presente' (Organização Didática, p. 23)."
    },
    {
        "pergunta": "Segundo o Art. 111, o aluno terá suas faltas registradas durante o período em que estiver em atendimento domiciliar?",
        "resposta_esperada": "Não. Conforme o parágrafo único do Art. 111, 'O aluno não terá suas faltas registradas, durante o período em que estiver sendo atendido em domicílio' (Organização Didática, p. 24)."
    },
    {
        "pergunta": "De acordo com o Art. 132, é vedado ao corpo docente usar ou atender o celular em sala de aula?",
        "resposta_esperada": "Sim. O Art. 132, inciso XV, veda ao corpo docente 'usar ou atender o celular em sala de aula ou quaisquer aparelhos eletrônicos que não estejam destinados ao processo de ensino-aprendizagem do aluno em situação de aula ou em momentos de avaliação' (Organização Didática, p. 30)."
    }
]

print(f"✓ Dataset com {len(evaluation_dataset)} perguntas carregado!")

✓ Dataset com 20 perguntas carregado!


---
## 6. Execução das Perguntas
**Execute esta célula para processar todas as perguntas do dataset**

In [ ]:
# Armazenar resultados
results = []

for idx, item in enumerate(evaluation_dataset, 1):
    print(f"\n\n{'#'*80}")
    print(f"PROCESSANDO PERGUNTA {idx}/{len(evaluation_dataset)}")
    print(f"{'#'*80}")

    generated_answer, retrieved_context = ask(
        item["pergunta"],
        rag_chain,
        retriever,
        cross_encoder,
        verbose=True
    )

    results.append({
        "pergunta": item["pergunta"],
        "resposta_esperada": item["resposta_esperada"],
        "resposta_gerada": generated_answer,
        "contexto_recuperado": retrieved_context,
    })

# Criar DataFrame com os resultados
df_results = pd.DataFrame(results)

print("\n\n" + "="*80)
print("✓ TODAS AS PERGUNTAS PROCESSADAS!")
print("="*80)
print(f"Total de perguntas: {len(results)}")



################################################################################
PROCESSANDO PERGUNTA 1/20
################################################################################

PERGUNTA: Quando o curso de Tecnologia em Análise e Desenvolvimento de Sistemas foi implantado no Campus de Picos?
✓ Recuperados 8 documentos

[DEBUG] Top 4 documentos após re-ranking:
  #1 (score: 7.1133): denominação: Análise e Desenvolvimento de Sistemas. Em 2002, foi autorizada a cr...
  #2 (score: 4.9391): O Instituto Federal do Piauí possui atualmente 20 campi distribuídos do norte ao...
  #3 (score: 4.6784): MINISTÉRIO DA EDUCAÇÃO Secretaria de Educação Profissional e Tecnológica Institu...
  #4 (score: 3.6548): de Redação do Vestibular/Exame Nacional do Ensino Médio (ENEM) em um dos últimos...

[DEBUG] Contexto LIMPO enviado ao LLM (primeiros 400 caracteres):
--------------------------------------------------------------------------------
[Trecho 1]: denominação: Análise e Desenvolvimento d

In [ ]:
# Visualizar resultados
print("\nPRÉVIA DOS RESULTADOS:\n")
for idx, row in df_results.iterrows():
    print(f"\n{'='*80}")
    print(f"PERGUNTA {idx+1}:")
    print(f"  {row['pergunta']}")
    print(f"\nESPERADA:")
    print(f"  {row['resposta_esperada']}")
    print(f"\nGERADA:")
    print(f"  {row['resposta_gerada']}")
    print("="*80)


PRÉVIA DOS RESULTADOS:


PERGUNTA 1:
  Quando o curso de Tecnologia em Análise e Desenvolvimento de Sistemas foi implantado no Campus de Picos?

ESPERADA:
  O curso foi implantado no Campus de Picos em 2013 (PPC de ADS - Picos, p. 17).

GERADA:
  O curso de Tecnologia em Análise e Desenvolvimento de Sistemas foi implantado no Campus de Picos, em 2013 (PPC de ADS - Picos, p. 17, 1, 22).

PERGUNTA 2:
  Quais são as modalidades aceitas para a apresentação do TCC?

ESPERADA:
  O TCC pode ser apresentado em formato de Monografia, Artigo Científico, Relatório Técnico de Software (RTS) ou Relatório Técnico de Trabalho/Estágio (RTT) (PPC de ADS - Picos, p. 75).

GERADA:
  As modalidades aceitas para a apresentação do TCC são: Monografia, Artigo Científico completo (com resultados) publicado em eventos regionais, nacionais e internacionais ou em periódicos pertencentes à Lista de Periódicos classificados no Qualis Capes ou na Revista Somma do IFPI. Além disso, os alunos também podem desenvolve

---
## 7. Avaliação de Métricas
**Execute esta seção após processar as perguntas para calcular as métricas**

In [ ]:
# Preparar dados para métricas
generated_answers = df_results["resposta_gerada"].tolist()
expected_answers = df_results["resposta_esperada"].tolist()

print(f"Preparando avaliação de {len(generated_answers)} respostas...")

Preparando avaliação de 20 respostas...


In [ ]:
# Carregar métricas
print("Carregando métricas de avaliação...\n")

rouge_metric = evaluate.load('rouge')
bleu_metric = evaluate.load('bleu')
bertscore_metric = evaluate.load('bertscore')

print("✓ Métricas carregadas!")

Carregando métricas de avaliação...

✓ Métricas carregadas!


In [ ]:
# Calcular ROUGE
print("Calculando ROUGE...")
rouge_scores = rouge_metric.compute(
    predictions=generated_answers,
    references=expected_answers
)
print("✓ ROUGE calculado!")

Calculando ROUGE...
✓ ROUGE calculado!


In [ ]:
# Calcular BLEU
print("Calculando BLEU...")
bleu_scores = bleu_metric.compute(
    predictions=generated_answers,
    references=expected_answers
)
print("✓ BLEU calculado!")

Calculando BLEU...
✓ BLEU calculado!


In [ ]:
# Calcular BERTScore (pode demorar)
print("Calculando BERTScore... (pode demorar)")
bertscore_scores = bertscore_metric.compute(
    predictions=generated_answers,
    references=expected_answers,
    lang='pt',
    model_type='distilbert-base-multilingual-cased'
)
print("✓ BERTScore calculado!")

Calculando BERTScore... (pode demorar)
✓ BERTScore calculado!


In [ ]:
# Calcular F1-Score
def calculate_f1_token(prediction, reference):
    pred_tokens = set(prediction.lower().split())
    ref_tokens = set(reference.lower().split())

    if len(pred_tokens) == 0 or len(ref_tokens) == 0:
        return 0.0

    common = pred_tokens.intersection(ref_tokens)
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(ref_tokens)

    if precision + recall == 0:
        return 0.0

    return 2 * (precision * recall) / (precision + recall)

print("Calculando F1-Score...")
f1_scores = [calculate_f1_token(p, r) for p, r in zip(generated_answers, expected_answers)]

avg_f1 = sum(f1_scores) / len(f1_scores)
avg_bertscore = sum(bertscore_scores['f1']) / len(bertscore_scores['f1'])

print("✓ F1-Scorecalculado!")

# Adicionar métricas ao DataFrame
df_results['f1_score'] = f1_scores

Calculando F1-Score...
✓ F1-Scorecalculado!


### 7.1 Visualização dos Resultados das Métricas

In [ ]:
# Exibir resultados completos
print("\n" + "="*80)
print("RESULTADOS COMPLETOS DAS MÉTRICAS")
print("="*80)

print("\n[ROUGE - Overlap de N-gramas]")
print(f"  ROUGE-1: {rouge_scores['rouge1']:.4f}")
print(f"  ROUGE-2: {rouge_scores['rouge2']:.4f}")
print(f"  ROUGE-L: {rouge_scores['rougeL']:.4f}")

print("\n[BLEU - Precisão de N-gramas]")
print(f"  BLEU Score: {bleu_scores['bleu']:.4f}")

print("\n[BERTScore - Similaridade Semântica]")
print(f"  F1 (média): {avg_bertscore:.4f}")
print(f"  Precision:  {sum(bertscore_scores['precision'])/len(bertscore_scores['precision']):.4f}")
print(f"  Recall:     {sum(bertscore_scores['recall'])/len(bertscore_scores['recall']):.4f}")

print("\n[F1-Score Token-based]")
print(f"  F1 (média): {avg_f1:.4f}")

print("\n" + "="*80)


RESULTADOS COMPLETOS DAS MÉTRICAS

[ROUGE - Overlap de N-gramas]
  ROUGE-1: 0.6517
  ROUGE-2: 0.5218
  ROUGE-L: 0.5915

[BLEU - Precisão de N-gramas]
  BLEU Score: 0.3562

[BERTScore - Similaridade Semântica]
  F1 (média): 0.9186
  Precision:  0.9065
  Recall:     0.9320

[F1-Score Token-based]
  F1 (média): 0.5840



In [ ]:
# Exibir métricas por pergunta e métricas gerais em tabela
print("\nMÉTRICAS DETALHADAS E POR PERGUNTA:\n")

# Criar DataFrame com as métricas gerais
metrics_data = {
    'Métrica': ['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'BLEU', 'BERTScore (F1 Médio)', 'F1-Score (Token Médio)'],
    'Valor': [
        rouge_scores['rouge1'],
        rouge_scores['rouge2'],
        rouge_scores['rougeL'],
        bleu_scores['bleu'],
        avg_bertscore,
        avg_f1,
    ]
}
df_metrics_general = pd.DataFrame(metrics_data)
print("\nMétricas Gerais:\n")
display(df_metrics_general.round(4))

# Adicionar BERTScore F1 por pergunta ao DataFrame de resultados
df_results['bertscore_f1'] = bertscore_scores['f1']
df_results['bertscore_precision'] = bertscore_scores['precision']
df_results['bertscore_recall'] = bertscore_scores['recall']


# Exibir métricas por pergunta
print("\nMétricas por Pergunta:\n")
display(df_results[['pergunta', 'f1_score', 'bertscore_f1', 'bertscore_precision', 'bertscore_recall']].round(4))


MÉTRICAS DETALHADAS E POR PERGUNTA:


Métricas Gerais:



,Métrica,Valor
0,ROUGE-1,0.6517
1,ROUGE-2,0.5218
2,ROUGE-L,0.5915
3,BLEU,0.3562
4,BERTScore (F1 Médio),0.9186
5,F1-Score (Token Médio),0.5840



Métricas por Pergunta:



,pergunta,f1_score,bertscore_f1,bertscore_precision,bertscore_recall
0,Quando o curso de Tecnologia em Análise e Dese...,0.7368,0.9431,0.9048,0.9847
1,Quais são as modalidades aceitas para a aprese...,0.4000,0.8950,0.8561,0.9377
2,Qual foi o montante total de investimento que ...,0.4681,0.9392,0.9085,0.9721
3,Qual o pré-requisito para cursar Programação p...,0.8095,0.9788,0.9679,0.9899
4,Quantas vagas anuais são ofertadas para o curs...,0.4186,0.8930,0.8816,0.9047
5,Qual é a carga horária total do curso de ADS e...,0.6197,0.8895,0.8747,0.9049
6,Quais são os dois laboratórios de pesquisa e e...,0.7458,0.9573,0.9427,0.9723
7,Como é calculada a nota final do Trabalho de C...,0.6531,0.9047,0.8899,0.9200
8,Qual disciplina é o pré-requisito necessário p...,0.8205,0.9600,0.9534,0.9666
9,"Em que ano a instituição, que hoje é o IFPI, f...",0.7200,0.9585,0.9454,0.9720


### 7.2 Análise Detalhada (Opcional)

In [ ]:
# Análise detalhada de cada resposta
print("\nANÁLISE DETALHADA DAS RESPOSTAS:\n")

for idx, row in df_results.iterrows():
    print(f"\n{'='*80}")
    print(f"PERGUNTA {idx+1}:")
    print(f"  {row['pergunta']}")
    print(f"\nRESPOSTA ESPERADA:")
    print(f"  {row['resposta_esperada']}")
    print(f"\nRESPOSTA GERADA:")
    print(f"  {row['resposta_gerada']}")
    print(f"\nMÉTRICAS:")
    print(f"  F1-Score: {row['f1_score']:.4f}")
    print(f"  BERTScore (F1): {row['bertscore_f1']:.4f}")
    print(f"  BERTScore (Precision): {row['bertscore_precision']:.4f}")
    print(f"  BERTScore (Recall): {row['bertscore_recall']:.4f}")
    print(f"  ROUGE-1: {rouge_scores['rouge1']:.4f}")
    print(f"  ROUGE-2: {rouge_scores['rouge2']:.4f}")
    print(f"  ROUGE-L: {rouge_scores['rougeL']:.4f}")
    print(f"  BLEU: {bleu_scores['bleu']:.4f}")
    print("="*80)


ANÁLISE DETALHADA DAS RESPOSTAS:


PERGUNTA 1:
  Quando o curso de Tecnologia em Análise e Desenvolvimento de Sistemas foi implantado no Campus de Picos?

RESPOSTA ESPERADA:
  O curso foi implantado no Campus de Picos em 2013 (PPC de ADS - Picos, p. 17).

RESPOSTA GERADA:
  O curso de Tecnologia em Análise e Desenvolvimento de Sistemas foi implantado no Campus de Picos, em 2013 (PPC de ADS - Picos, p. 17, 1, 22).

MÉTRICAS:
  F1-Score: 0.7368
  BERTScore (F1): 0.9431
  BERTScore (Precision): 0.9048
  BERTScore (Recall): 0.9847
  ROUGE-1: 0.6517
  ROUGE-2: 0.5218
  ROUGE-L: 0.5915
  BLEU: 0.3562

PERGUNTA 2:
  Quais são as modalidades aceitas para a apresentação do TCC?

RESPOSTA ESPERADA:
  O TCC pode ser apresentado em formato de Monografia, Artigo Científico, Relatório Técnico de Software (RTS) ou Relatório Técnico de Trabalho/Estágio (RTT) (PPC de ADS - Picos, p. 75).

RESPOSTA GERADA:
  As modalidades aceitas para a apresentação do TCC são: Monografia, Artigo Científico completo (

---
## 8. Salvamento dos Resultados

In [ ]:
# Salvar resultados em CSV
output_filename = "resultados_rag_llama3.csv"
df_results.to_csv(output_filename, index=False)
print(f"✓ Resultados salvos em '{output_filename}'")

In [ ]:
# Salvar resumo das métricas
metrics_summary = {
    'ROUGE-1': rouge_scores['rouge1'],
    'ROUGE-2': rouge_scores['rouge2'],
    'ROUGE-L': rouge_scores['rougeL'],
    'BLEU': bleu_scores['bleu'],
    'BERTScore_F1': avg_bertscore,
    'BERTScore_Precision': sum(bertscore_scores['precision'])/len(bertscore_scores['precision']),
    'BERTScore_Recall': sum(bertscore_scores['recall'])/len(bertscore_scores['recall']),
    'F1_Token': avg_f1,
    'Total_Perguntas': len(df_results)
}

df_summary = pd.DataFrame([metrics_summary])
summary_filename = "metricas_resumo.csv"
df_summary.to_csv(summary_filename, index=False)
print(f"✓ Resumo das métricas salvo em '{summary_filename}'")

✓ Resumo das métricas salvo em 'metricas_resumo.csv'


---
## 9. Teste Rápido (Pergunta Única)
**Use esta célula para testar uma pergunta específica sem avaliar métricas**

In [ ]:
# Teste rápido com uma pergunta
pergunta_teste = "Qual o pré-requisito para cursar Programação para Dispositivos Móveis?"

resposta, contexto = ask(
    pergunta_teste,
    rag_chain,
    retriever,
    cross_encoder,
    verbose=True
)

---
## 10. Limpeza de Memória (Opcional)
**Execute se precisar liberar recursos da GPU**

In [ ]:
def clear_gpu_memory(*args):
    """Libera a memória da GPU."""
    print("\n--- Limpando a memória da GPU ---")
    for arg in args:
        try:
            del arg
        except NameError:
            pass
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("✓ Cache da GPU limpo.")

# Descomente para limpar memória
# clear_gpu_memory(llm, embeddings, vectorstore, cross_encoder)

---
## 11. Executar Múltiplas Rodadas de Avaliação
**Execute esta seção para rodar o dataset N vezes e obter estatísticas agregadas**

In [ ]:
# Configurar número de rodadas
NUM_RODADAS = 10  # Altere para quantas rodadas desejar

print(f"Configurado para executar {NUM_RODADAS} rodadas de avaliação")

Configurado para executar 10 rodadas de avaliação


In [ ]:
# Executar múltiplas rodadas
all_rounds_results = []
all_rounds_metrics = []

for rodada in range(1, NUM_RODADAS + 1):
    print(f"\n\n{'#'*80}")
    print(f"RODADA {rodada}/{NUM_RODADAS}")
    print(f"{'#'*80}\n")

    rodada_results = []

    for idx, item in enumerate(evaluation_dataset, 1):
        print(f"\n[Rodada {rodada}] Pergunta {idx}/{len(evaluation_dataset)}")

        generated_answer, retrieved_context = ask(
            item["pergunta"],
            rag_chain,
            retriever,
            cross_encoder,
            verbose=True
        )

        # FIX: Append to rodada_results instead of results
        rodada_results.append({
            "pergunta": item["pergunta"],
            "resposta_esperada": item["resposta_esperada"],
            "resposta_gerada": generated_answer,
            "contexto_recuperado": retrieved_context,
        })

    # Calcular métricas para esta rodada
    gen_ans = [r["resposta_gerada"] for r in rodada_results]
    exp_ans = [r["resposta_esperada"] for r in rodada_results]

    # Ensure lists are not empty before computing metrics
    if gen_ans and exp_ans:
        rouge = rouge_metric.compute(predictions=gen_ans, references=exp_ans)
        bleu = bleu_metric.compute(predictions=gen_ans, references=exp_ans)
        bertscore = bertscore_metric.compute(
            predictions=gen_ans, references=exp_ans,
            lang='pt', model_type='distilbert-base-multilingual-cased'
        )

        f1_round = [calculate_f1_token(p, r) for p, r in zip(gen_ans, exp_ans)]

        rodada_metrics = {
            'rodada': rodada,
            'ROUGE-1': rouge['rouge1'],
            'ROUGE-2': rouge['rouge2'],
            'ROUGE-L': rouge['rougeL'],
            'BLEU': bleu['bleu'],
            'BERTScore_F1': sum(bertscore['f1']) / len(bertscore['f1']),
            'F1_Token': sum(f1_round) / len(f1_round),
        }
        all_rounds_metrics.append(rodada_metrics)
    else:
        print(f"Skipping metric calculation for round {rodada} due to empty results.")
        rodada_metrics = {
            'rodada': rodada,
            'ROUGE-1': 0.0, 'ROUGE-2': 0.0, 'ROUGE-L': 0.0, 'BLEU': 0.0,
            'BERTScore_F1': 0.0, 'F1_Token': 0.0
        }
        all_rounds_metrics.append(rodada_metrics)


    all_rounds_results.extend(rodada_results)


    print(f"\n✓ Rodada {rodada} concluída!")
    print(f"  F1-Token: {rodada_metrics['F1_Token']:.4f}")

print(f"\n\n{'='*80}")
print(f"✓ TODAS AS {NUM_RODADAS} RODADAS CONCLUÍDAS!")
print("="*80)



################################################################################
RODADA 1/10
################################################################################


[Rodada 1] Pergunta 1/20

PERGUNTA: Quando o curso de Tecnologia em Análise e Desenvolvimento de Sistemas foi implantado no Campus de Picos?
✓ Recuperados 8 documentos

[DEBUG] Top 4 documentos após re-ranking:
  #1 (score: 7.1133): denominação: Análise e Desenvolvimento de Sistemas. Em 2002, foi autorizada a cr...
  #2 (score: 4.9391): O Instituto Federal do Piauí possui atualmente 20 campi distribuídos do norte ao...
  #3 (score: 4.6784): MINISTÉRIO DA EDUCAÇÃO Secretaria de Educação Profissional e Tecnológica Institu...
  #4 (score: 3.6548): de Redação do Vestibular/Exame Nacional do Ensino Médio (ENEM) em um dos últimos...

[DEBUG] Contexto LIMPO enviado ao LLM (primeiros 400 caracteres):
--------------------------------------------------------------------------------
[Trecho 1]: denominação: Análise e Dese

In [ ]:
# Análise agregada das rodadas
df_all_rounds = pd.DataFrame(all_rounds_results)
df_metrics_rounds = pd.DataFrame(all_rounds_metrics)

print("\nESTATÍSTICAS AGREGADAS DE TODAS AS RODADAS:\n")
# Display describe() output as a DataFrame
display(df_metrics_rounds.describe().round(4))

print("\n\nMÉDIAS FINAIS (todas as rodadas):\n")
final_averages_data = {}
for col in df_metrics_rounds.columns:
    if col != 'rodada':
        mean_val = df_metrics_rounds[col].mean()
        std_val = df_metrics_rounds[col].std()
        final_averages_data[col] = [mean_val, std_val]

df_final_averages = pd.DataFrame(final_averages_data, index=['Média', 'Desvio Padrão'])
display(df_final_averages.round(4))


# Calcular e exibir médias por rodada
print("\n\nMÉTRICAS MÉDIAS POR RODADA:\n")
# Ensure the relevant columns exist before grouping
metrics_cols = ['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'BLEU', 'BERTScore_F1', 'F1_Token']
df_metrics_rounds_grouped = df_metrics_rounds.groupby('rodada')[metrics_cols].mean()
display(df_metrics_rounds_grouped.round(4))


ESTATÍSTICAS AGREGADAS DE TODAS AS RODADAS:



,rodada,ROUGE-1,ROUGE-2,ROUGE-L,BLEU,BERTScore_F1,F1_Token
count,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000
mean,5.5000,0.6709,0.5271,0.5961,0.3771,0.9200,0.5956
std,3.0277,0.0130,0.0122,0.0106,0.0199,0.0027,0.0093
min,1.0000,0.6485,0.5142,0.5834,0.3415,0.9163,0.5794
25%,3.2500,0.6630,0.5159,0.5882,0.3638,0.9177,0.5901
50%,5.5000,0.6709,0.5272,0.5942,0.3790,0.9205,0.5953
75%,7.7500,0.6797,0.5321,0.6012,0.3913,0.9217,0.6007
max,10.0000,0.6930,0.5521,0.6173,0.4087,0.9245,0.6128




MÉDIAS FINAIS (todas as rodadas):



,ROUGE-1,ROUGE-2,ROUGE-L,BLEU,BERTScore_F1,F1_Token
Média,0.6709,0.5271,0.5961,0.3771,0.9200,0.5956
Desvio Padrão,0.0130,0.0122,0.0106,0.0199,0.0027,0.0093




MÉTRICAS MÉDIAS POR RODADA:



,ROUGE-1,ROUGE-2,ROUGE-L,BLEU,BERTScore_F1,F1_Token
rodada,,,,,,
1,0.6815,0.5317,0.5958,0.3916,0.9217,0.6027
2,0.6666,0.5154,0.5834,0.3649,0.9200,0.5955
3,0.6617,0.5176,0.5926,0.3634,0.9189,0.5928
4,0.6683,0.5255,0.5860,0.3830,0.9211,0.5951
5,0.6829,0.5323,0.5984,0.3930,0.9223,0.6008
6,0.6734,0.5383,0.6022,0.3904,0.9214,0.6003
7,0.6485,0.5149,0.5876,0.3415,0.9163,0.5794
8,0.6584,0.5142,0.5901,0.3600,0.9168,0.5872
9,0.6742,0.5290,0.6079,0.3750,0.9172,0.5892


In [ ]:
# Salvar resultados de múltiplas rodadas
df_all_rounds.to_csv("resultados_multiplas_rodadas.csv", index=False)
df_metrics_rounds.to_csv("metricas_multiplas_rodadas.csv", index=False)

print("✓ Resultados de múltiplas rodadas salvos!")
print("  - resultados_multiplas_rodadas.csv")
print("  - metricas_multiplas_rodadas.csv")

✓ Resultados de múltiplas rodadas salvos!
  - resultados_multiplas_rodadas.csv
  - metricas_multiplas_rodadas.csv


---
## 📝 Instruções de Uso

### Fluxo Recomendado:

1. **Primeira Execução (Setup Completo):**
   - Execute seções 1 e 2 (Imports + Construção do Índice) - APENAS 1 VEZ
   - Execute seção 3 (Carregamento do Sistema RAG)
   - Execute seção 4 (Funções)

2. **Avaliações Subsequentes:**
   - Pule a seção 2 (índice já existe)
   - Execute seção 3 (Carregar sistema)
   - Execute seção 4 (Funções)
   - Configure dataset na seção 5
   - Execute seção 6 (Processar perguntas)
   - Execute seção 7 (Calcular métricas)

3. **Para Múltiplas Rodadas:**
   - Configure NUM_RODADAS na seção 11
   - Execute toda a seção 11 para análise estatística

4. **Teste Rápido:**
   - Use seção 9 para testar perguntas individuais sem avaliação formal

### Dicas:
- Os modelos são carregados **uma vez** e reutilizados
- Adicione quantas perguntas quiser na seção 5
- As métricas são calculadas de forma **independente** das perguntas
- Use a seção 10 se precisar liberar memória GPU entre execuções